# INIT

In [ ]:
import rospy
import inspect
from robotblockset.ros.grippers_ros import PandaGripper
from importlib import reload
import example_data_recorder
#reload(example_data_recorder)
from utils import SoftSetJointCompliance
ns = "pingvin_2"
rospy.init_node(ns)
from robotblockset.ros.franka import panda_ros
r = panda_ros(ns=ns, control_strategy="JointImpedance", init_node=False)
g = PandaGripper(namespace=ns,robot=r)

In [ ]:
r.ErrorRecovery()

# Record positional trajectory
### @ low stiffness

### Robot guidance 

In [ ]:
import time
def SoftSetJointCompliance(robot, target_k, tm):
    # Get the current joint stiffness values
    k0 = robot.joint_compliance.K
    
    # Calculate the number of steps
    N = max(1, round(tm / robot.tsamp))
    
    # Calculate the incremental changes 
    dK = (target_k - k0) / N

    #initialize
    Ki = k0
    target_d = robot._franka_default.JointCompliance.D

    robot.controller._verbose = 0
    # Gradually update compliance
    for i in range(1, N):
        Ki += dK
        robot.SetJointCompliance(Ki,target_d)
        time.sleep(robot.tsamp)
        
    robot.controller._verbose = 1
    robot.SetJointCompliance(Ki+dK,target_d)

In [ ]:
import numpy as np
import time
def record_robot_state(robot, frequency, duration):
    interval = 1.0 / frequency
    
    N=int(frequency*duration)
    tt = np.zeros((N,1))
    qt = np.zeros((N,r.nj))
    dqt = np.zeros((N,r.nj))
    xt = np.zeros((N,r.nj))
    trq = np.zeros((N,r.nj))

    i = 0
    start_time = time.monotonic()
    end_time = start_time + duration
    
    last_update = time.monotonic()

    while time.monotonic() <= end_time+interval and i < N:
        t = time.monotonic()
        if t - last_update >= interval:
            
            # Get robot state
            state = robot.GetState()
            xt[i] = robot.x
            tt[i] = t-start_time
            qt[i] = robot.q
            dqt[i] = robot.qdot
            #trq[i] = np.array(r.state.tau_J_d) #new line to store trqs

            # Update the last recorded time
            last_update = t
            i += 1
        
    return xt, tt, qt, dqt, i, N#, trq 

In [ ]:
#Set low stiff
r.SetJointCompliant()

In [ ]:
import time
from utils import pip

time.sleep(2)
pip();
xt, tt, qt, dqt, i, N = record_robot_state(r,100,5)
pip();

In [ ]:
r.ErrorRecovery()

In [ ]:
r.ResetCurrentTarget()
SoftSetJointCompliance(r,r._franka_default.JointCompliance.K,3);

### Encode as DMP

In [ ]:
from dmp import DMP

d = DMP(qt.copy(),np.squeeze(tt.copy()),vel_data=dqt.copy(),num_weights=25)

trj_dec,t_dec=d.decode()

In [ ]:
from robotblockset.graphics import plotjtraj
import matplotlib.pyplot as plt

plotjtraj(qt=trj_dec, t=t_dec);

# Record torques
### @ high stiffness

### Execute recorded positional trj

In [ ]:
#Set high stiff
r.ResetCurrentTarget()
#SoftSetJointCompliance(r,r._franka_default.JointCompliance.K,3);
r.SetJointStiff()

In [ ]:
# Move slowly to the first configuration
r.JMove(trj_dec[0],5)
trq = np.zeros_like(trj_dec)
pip();
i = 0;
for q in trj_dec: 
    # Call the GoTo_q function
    r.GoTo_q(q,np.zeros(7),np.zeros(7),0)
    trq[i] = np.array(r.state.tau_J_d) #new line to store trqs
    i += 1
    time.sleep(1/100.)

### Encode as TP

In [ ]:
len(trq)
trq.shape

In [ ]:
#pad with zeros
trqz = (np.vstack((np.zeros((1,7)),trq)))
len(trqz)
trqz.shape


In [ ]:
plt.plot(tt,trqz)

In [ ]:
#test TP
from tp import TP

t = TP(trqz.copy(),np.squeeze(tt.copy()),num_weights=50)
trq_dec, t_dec = t.decode()
plt.plot(t_dec,trq_dec)

# Execute CMP
### @ low stiffness

### Test at low stiff without TP

In [ ]:
#Set high stiff
r.ResetCurrentTarget()
SoftSetJointCompliance(r,r._franka_default.JointCompliance.K,3);

In [ ]:
# Move slowly to the first configuration
r.JMove(trj_dec[0],3)

In [ ]:
#Set low stiff
#r.SetJointCompliant()
r.SetJointCompliance([5., 5., 5., 5., 3., 3., 1.],3);
#SoftSetJointCompliance(r,[5., 5., 5., 5., 2., 2., 1.],3);

In [ ]:
#Execute JUST pos at low stiff
pip();
i = 0
for q in trj_dec: 
    # Call the GoTo_q function
    r.GoTo_q(q,np.zeros(7),np.zeros(7),0)
    #print(trq_dec[i])
    i += 1
    time.sleep(1/100.)

pip();

In [ ]:
#Set high stiff
r.ResetCurrentTarget()
SoftSetJointCompliance(r,r._franka_default.JointCompliance.K,3);

### Execute CMP

# Set high stiff
r.ResetCurrentTarget()
SoftSetJointCompliance(r,r._franka_default.JointCompliance.K,3);
r.SetJoitStiff()

In [ ]:
# Move slowly to the first configuration
r.JMove(trj_dec[0],3)

In [ ]:
#Set low stiff
#r.SetJointCompliant()
r.SetJointCompliance([5., 5., 5., 5., 3., 3., 1.],3);
#SoftSetJointCompliance(r,[5., 5., 5., 5., 2., 2., 1.],3);

In [ ]:
#Execute JUST pos at low stiff
pip();
i = 0
for q in trj_dec: 
    # Call the GoTo_q function
    r.GoTo_q(q,np.zeros(7),trq_dec[i],0)
    #print(trq_dec[i])
    i += 1
    time.sleep(1/100.)

pip();

In [ ]:
#Set high stiff
r.ResetCurrentTarget()
SoftSetJointCompliance(r,r._franka_default.JointCompliance.K,3);

# Gripper test

In [ ]:
r.ErrorRecovery()

In [ ]:
r.SetJointCompliant()

In [ ]:
g.Homing()

In [ ]:
r.SetJointCompliant()

In [ ]:
r.ResetCurrentTarget()
SoftSetJointCompliance(r,r._franka_default.JointCompliance.K,3);
r.SetJointStiff()

In [ ]:
r.SetJointCompliance([1200.,1200.,1200.,1200.,1200.,1200.,250.],3);

In [ ]:
r.CMoveFor([0,0,0.1],3)

In [ ]:
g.Close()

In [ ]:
R.c

In [ ]:
g.Open()

In [ ]:
r.SetJointCompliant()